<a href="https://colab.research.google.com/github/Dobarri/22URP_kaist_dataset/blob/thermal_1/th_kaist_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import torch
from tqdm import tqdm

def parse_annotation (annotation_path) :
    
    with open(annotation_path, 'r') as f :
        json_data = json.load(f)

    boxes = []
    labels = []
    difficulties = []

    json_annotations = json_data.get("annotation")

    for annotation in json_annotations :

        #label
        json_label = annotation.get("category_id")
        if json_label == -1 :
            json_label = 0
        labels.append(json_label)
        
        #difficulties
        difficult = int(annotation.get('is_crowd') == '1')
        difficulties.append(difficult)

        #bbox
        json_boxes = annotation.get("bbox")
        xmin = json_boxes[0]
        ymin = json_boxes[1]
        xmax = json_boxes[2]
        ymax = json_boxes[3]

        boxes.append([xmin, ymin, xmax, ymax])


    return {'boxes': boxes, 'labels': labels, 'difficulties': difficulties}      
        
def create_data_lists (kaist_data_path, output_folder) :
    data_path = os.path.abspath(kaist_data_path) 

    train_paths = []
    train_images = []
    test_paths = []
    test_images = []
    n_objects = 0

    #training data
    train_data_path = os.path.join(data_path,'train-all-20.txt')
        
    #training object path
    with open (train_data_path) as f :
        ids = f.read().splitlines()

    for id in tqdm(ids) :
        #train_images
        train_paths.append(os.path.join(data_path, 'images', id.split('/')[0], id.split('/')[1], 'lwir', id.split('/')[2] + '.jpg'))

        objects = parse_annotation(os.path.join(data_path, 'annotation_json', id + '.json'))
        if len(objects['boxes']) == 0:
                continue
        train_images.append(objects)

    
    save_train_image = "/content/drive/MyDrive/th_KAISTSSD/th_train_objects.json"
    save_train_path = "/content/drive/MyDrive/th_KAISTSSD/th_train_images.json"
    
    with open(save_train_image, 'w', encoding='utf-8') as file:
        json.dump(train_images, file, indent = "\t")
    with open(save_train_path, 'w', encoding='utf-8') as file:
        json.dump(train_paths, file, indent = "\t")

    print('\nThere are %d training images.' % (len(train_images)))


    #test data
    test_data_path = os.path.join(data_path,'test-all-20.txt')

    #test object path
    with open (test_data_path) as f :
        ids = f.read().splitlines()
        
    for id in tqdm(ids) :
        #test_images
        test_paths.append(os.path.join(data_path, 'images', id.split('/')[0], id.split('/')[1], 'lwir', id.split('/')[2] + '.jpg'))

        objects = parse_annotation(os.path.join(data_path, 'annotation_json', id + '.json'))
        if len(objects['boxes']) == 0:
                continue
        test_images.append(objects)
    
    save_test_image = "/content/drive/MyDrive/th_KAISTSSD/th_test_objects.json"
    save_test_path = "/content/drive/MyDrive/th_KAISTSSD/th_test_images.json"
    
    with open(save_test_image, 'w', encoding='utf-8') as file:
        json.dump(test_images, file, indent = "\t")
    with open(save_test_path, 'w', encoding='utf-8') as file:
        json.dump(test_paths, file, indent = "\t")

    print('\nThere are %d training images.' % (len(test_images)))


In [3]:
create_data_lists ('/content/drive/MyDrive/KAIST_PD', './')

100%|██████████| 8730/8730 [1:25:13<00:00,  1.71it/s]



There are 8730 training images.


100%|██████████| 2252/2252 [23:24<00:00,  1.60it/s]



There are 2252 training images.


In [4]:
%cd /content/drive/MyDrive/th_KAISTSSD

/content/drive/MyDrive/kaistSSD


In [6]:
!python thermal_train.py

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Loaded base model.

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
Epoch: [0][0/273]	Batch Time 47.065 (47.065)	Data Time 32.710 (32.710)	Loss 18.1515 (18.1515)	

In [7]:
!python th_kaist_detect.py


Loaded checkpoint from epoch 80.

  0% 3/2252 [00:07<1:12:02,  1.92s/it]/content/drive/MyDrive/kaistSSD/model.py:503: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/drive/MyDrive/kaistSSD/model.py:505: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  image_scores.append(class_scores[1 - suppress])
100% 2252/2252 [21:25<00:00,  1.75it/s]


In [15]:
%cd /content/drive/MyDrive/2022.URP.FallSemester/Week.5/official_Evaluation

/content/drive/MyDrive/2022.URP.FallSemester/Week.5/official_Evaluation


In [16]:
!pwd

/content/drive/MyDrive/2022.URP.FallSemester/Week.5/official_Evaluation


In [18]:
!python Evaluation_official.py

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.54s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.17s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 52.54%
Recall: 0.6127819548872181
